<a href="https://colab.research.google.com/github/forouzanfallah/Search_ArXiv_for_Papers/blob/main/Search_ArXiv_for_Papers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import requests
from bs4 import BeautifulSoup
import datetime
import re

def search_arxiv_cs(keyword, start_date, end_date):
    base_url = "https://arxiv.org"
    search_url = f"{base_url}/search/cs?query={keyword}&searchtype=all&abstracts=show&order=-announced_date_first&size=50"
    papers = []

    # Send request to the arXiv search page
    response = requests.get(search_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all the paper entries
        entries = soup.find_all('li', class_='arxiv-result')
        for entry in entries:
            # Extract date, title, link, and abstract
            date_text = entry.find('p', class_='is-size-7').text.strip()
            match = re.search(r'originally announced (\w+) (\d{4}).', date_text)
            if match:
                month, year = match.groups()
                date = datetime.datetime.strptime(f"{month} {year}", '%B %Y')

                # Filter by date
                if start_date <= date <= end_date:
                    title = entry.find('p', class_='title is-5 mathjax').text.strip()
                    identifier = entry.find('p', class_='list-title is-inline-block').a.text.strip()
                    identifier = identifier.replace('arXiv:', '').strip()
                    link = f"https://arxiv.org/abs/{identifier}"
                    abstract = entry.find('span', class_='abstract-full has-text-grey-dark mathjax').text.strip()
                    abstract = re.sub(r'\s+', ' ', abstract)  # Replace multiple whitespace with single space

                    papers.append({'title': title, 'date': date, 'link': link, 'abstract': abstract})
    return papers


In [10]:
# Example usage
keyword = "Diffusion Models"
start_date = datetime.datetime(2024, 1, 1)
end_date = datetime.datetime(2024, 1, 31)
search_results = search_arxiv_cs(keyword, start_date, end_date)

# Display the first few results
if search_results:
    for paper in search_results[:5]:
        print(f"Title: {paper['title']}\nDate: {paper['date']}\nLink: {paper['link']}\nAbstract:")
        sentences = re.split(r'(?<=[.!?]) +', paper['abstract'])
        for sentence in sentences:
            print(sentence)
        print("\n")
else:
    print("No results found.")

Title: Synthesizing Moving People with 3D Control
Date: 2024-01-01 00:00:00
Link: https://arxiv.org/abs/2401.10889
Abstract:
In this paper, we present a diffusion model-based framework for animating people from a single image for a given target 3D motion sequence.
Our approach has two core components: a) learning priors about invisible parts of the human body and clothing, and b) rendering novel body poses with proper clothing and texture.
For the first part, we learn an in-filling diffusion model to hallucinate unseen parts of a person given a single image.
We train this model on texture map space, which makes it more sample-efficient since it is invariant to pose and viewpoint.
Second, we develop a diffusion-based rendering pipeline, which is controlled by 3D human poses.
This produces realistic renderings of novel poses of the person, including clothing, hair, and plausible in-filling of unseen regions.
This disentangled approach allows our method to generate a sequence of images th